# nvImageCodecs examples

In [ ]:
import os
import cv2
from matplotlib import pyplot as plt

In [ ]:
resources_dir = "../../resources"

Create Decoder and Encoder

In [ ]:
from nvidia import nvimgcodecs
decoder = nvimgcodecs.Decoder()
encoder = nvimgcodecs.Encoder()

Load and decode Jpeg2000 image with nvImageCodecs

In [ ]:
with open(resources_dir + "/base/cat.j2k", 'rb') as in_file:
    data = in_file.read()
    nv_img_cat = decoder.decode(data)

Save image to bmp file with nvImageCodecs

In [ ]:
with open("cat-j2k-o.bmp", 'wb') as out_file:
    data = encoder.encode(nv_img_cat, "bmp")
    out_file.write(data)

Read back with OpenCV just saved (with nvImageCodecs) bmp image 

In [ ]:
cv_img_4k_bmp = cv2.imread("cat-j2k-o.bmp")
cv_img_4k_bmp = cv2.cvtColor(cv_img_4k_bmp, cv2.COLOR_BGR2RGB)
plt.imshow(cv_img_4k_bmp)

Load and decode 4k Jpeg2000 (in jp2 container) image with nvImageCodecs in one function

In [ ]:
nv_img4k = decoder.read(resources_dir + "/base/4k_lossless.jp2")

Save 4k image to jpg file with nvImageCodecs

In [ ]:
enc_params = nvimgcodecs.EncodeParams()
encoder.write("4k_lossless-jp2-o.jpg", nv_img4k)

Read back with OpenCV just save (with nvImageCodecs) bmp image 

In [ ]:
image = cv2.imread("4k_lossless-jp2-o.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

Load jpg with nvImageCodecs

In [ ]:
nv_img_jpg = decoder.read(resources_dir + "/base/cat_q95_444.jpg")

Save as Jpeg 2000 with nvImageCodecs

In [ ]:
encoder.write("cat-jpg-o.j2k", nv_img_jpg)

Read back with OpenCV just saved (with nvImageCodecs) j2k image 

In [ ]:
image = cv2.imread("cat-jpg-o.j2k")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

### Passing decoding parameters

Decode jpeg with Exif orientation - by default it applies exif orientation

In [ ]:
nv_img_jpg = decoder.read(resources_dir+ "/base/f8-exif.jpg")
encoder.write("f8-exif.bmp", nv_img_jpg)
image = cv2.imread("f8-exif.bmp")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

Let assume we would like to ignore exif orientation

In [ ]:
dec_params = nvimgcodecs.DecodeParams(apply_exif_orientation=False)
nv_img_jpg = decoder.read(resources_dir + "/base/f8-exif.jpg", dec_params)
encoder.write("f8-wo-exif.bmp", nv_img_jpg)
image = cv2.imread("f8-wo-exif.bmp")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

### Passing encoding parameters

Changing quality and chroma subsampling in jpeg

In [ ]:
nv_img_jpg = decoder.read(resources_dir + "/base/cat_q95_444.jpg")
enc_params = nvimgcodecs.EncodeParams(quality=5, chroma_subsampling=nvimgcodecs.ChromaSubsampling.CSS_GRAY)
encoder.write("cat-q5-gray.jpg", nv_img_jpg, params=enc_params)
                    
image = cv2.imread("cat-q5-gray.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

Jpeg optimized huffman and progressive encoding

In [ ]:
nv_img_jpg = decoder.read(resources_dir + "/base/cat_q95_444.jpg")
encoder.write("cat-q75.jpg", nv_img_jpg, params=nvimgcodecs.EncodeParams(quality=75))
encoder.write("cat-q75-optimized_huffman.jpg", nv_img_jpg, params=nvimgcodecs.EncodeParams(
    quality=75, jpeg_optimized_huffman=True, jpeg_progressive = True))

print("default huffman file size:", os.path.getsize("cat-q75.jpg"))
print("optimized huffman file size:", os.path.getsize(
    "cat-q75-optimized_huffman.jpg"))

image = cv2.imread("cat-q75-optimized_huffman.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)


Encode lossless and lossy with jpeg2000

In [ ]:
import ctypes
nv_img_jpg = decoder.read(resources_dir + "/base/cat_q95_444.jpg")

encoder.write("cat-psnr20.j2k", nv_img_jpg, params=nvimgcodecs.EncodeParams(target_psnr=20.90))
encoder.write("cat-lossless.j2k", nv_img_jpg,
               params=nvimgcodecs.EncodeParams(jpeg2k_reversible=True))
encoder.write("cat-lossless-2decomps.j2k", nv_img_jpg,
               params=nvimgcodecs.EncodeParams(
                  jpeg2k_reversible=True, 
                  jpeg2k_num_resolutions=2, 
                  jpeg2k_code_block_size=(32, 32), 
                  jpeg2k_bitstream_type=nvimgcodecs.Jpeg2kBitstreamType.JP2,
                  jpeg2k_prog_order=nvimgcodecs.Jpeg2kProgOrder.LRCP))
print("lossy file size:", os.path.getsize("cat-psnr20.j2k"))
print("lossless file size:", os.path.getsize("cat-lossless.j2k"))
print("lossless 2 decomposition levels file size:",
      os.path.getsize("cat-lossless-2decomps.j2k"))
image = cv2.imread("cat-psnr20.j2k")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

We can specify allowed backends used for decoding

In [ ]:
gpu_dec = nvimgcodecs.Decoder(backends=[nvimgcodecs.Backend(nvimgcodecs.GPU_ONLY, load_hint=0.5), nvimgcodecs.Backend(nvimgcodecs.HYBRID_CPU_GPU)])
cpu_dec = nvimgcodecs.Decoder(backend_kinds=[nvimgcodecs.CPU_ONLY])

In [ ]:
%%time
nv_img_j2k = cpu_dec.read(resources_dir + "/base/4k_lossless.jp2")

In [ ]:
%%time
nv_img_j2k = gpu_dec.read(resources_dir + "/base/4k_lossless.jp2")

The same way we can create Encoder with allowed backends.

In [ ]:
gpu_enc = nvimgcodecs.Encoder(backends=[nvimgcodecs.Backend(nvimgcodecs.GPU_ONLY, load_hint=0.5), nvimgcodecs.Backend(nvimgcodecs.HYBRID_CPU_GPU)])
cpu_enc = nvimgcodecs.Encoder(backend_kinds=[nvimgcodecs.CPU_ONLY])

In [ ]:
gpu_enc.write("4k_lossless_gpu_out.jpg", nv_img_j2k)

Currently there is no CPU encoder available for jpeg so having cpu_enc we can write for example to bmp

In [ ]:
nv_img_j2k = cpu_enc.write("4k_lossless_cpu_out.bmp", nv_img_j2k)

### nvImageCodes Image  __cuda_array_interface__ support

In [ ]:
print(nv_img4k.__cuda_array_interface__)
print(nv_img4k.shape)